In [ ]:
import os
import Cell_BLAST as cb
import utils
os.environ["CUDA_VISIBLE_DEVICES"] = utils.pick_gpu_lowest_memory()
cb.config.RANDOM_SEED = 0
cb.config.N_JOBS = 4
fixed_model_kwargs = dict(
    latent_dim=10, cat_dim=20,
    epoch=500, patience=20
)

In [ ]:
cb.__version__

---
# Mouse
## Quake_10x_Tongue

In [ ]:
quake_10x_tongue = cb.data.ExprDataSet.read_dataset("../../Datasets/data/Quake_10x_Tongue/data.h5")
utils.peek(quake_10x_tongue, "build/tongue/Quake_10x_Tongue")
quake_10x_tongue.obs.head()

In [ ]:
quake_10x_tongue.obs["cluster"] = "cluster_" + quake_10x_tongue.obs["cluster"].astype(int).astype(str)
quake_10x_tongue.obs.dtypes

In [ ]:
opt_model_kwargs = dict(batch_effect="donor")
quake_10x_tongue_model = cb.directi.fit_DIRECTi(
    quake_10x_tongue, quake_10x_tongue.uns["seurat_genes"],
    **fixed_model_kwargs, **opt_model_kwargs
)
quake_10x_tongue.latent = quake_10x_tongue_model.inference(quake_10x_tongue)

In [ ]:
ax = quake_10x_tongue.visualize_latent("cell_ontology_class", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/tongue/Quake_10x_Tongue/cell_ontology_class.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = quake_10x_tongue.visualize_latent("cell_type1", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/tongue/Quake_10x_Tongue/cell_type1.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = quake_10x_tongue.visualize_latent("cluster", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/tongue/Quake_10x_Tongue/cluster.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = quake_10x_tongue.visualize_latent("free_annotation", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/tongue/Quake_10x_Tongue/free_annotation.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = quake_10x_tongue.visualize_latent("donor", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/tongue/Quake_10x_Tongue/donor.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = quake_10x_tongue.visualize_latent("gender", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/tongue/Quake_10x_Tongue/gender.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
quake_10x_tongue.write_dataset("build/tongue/Quake_10x_Tongue/Quake_10x_Tongue.h5")

In [ ]:
%%capture capio
quake_10x_tongue_models = [quake_10x_tongue_model]
opt_model_kwargs = dict(batch_effect="donor")
for i in range(1, cb.config.N_JOBS):
    print("==== Model %d ====" % i)
    quake_10x_tongue_models.append(cb.directi.fit_DIRECTi(
        quake_10x_tongue, quake_10x_tongue.uns["seurat_genes"],
        **fixed_model_kwargs, **opt_model_kwargs,
        random_seed=i
    ))
quake_10x_tongue_blast = cb.blast.BLAST(quake_10x_tongue_models, quake_10x_tongue)
quake_10x_tongue_blast.save("build/tongue/Quake_10x_Tongue")

In [ ]:
with open("build/tongue/Quake_10x_Tongue/stdout.txt", "w") as f:
    f.write(capio.stdout)
with open("build/tongue/Quake_10x_Tongue/stderr.txt", "w") as f:
    f.write(capio.stderr)

In [ ]:
utils.self_projection(quake_10x_tongue_blast, "build/tongue/Quake_10x_Tongue")

In [ ]:
%%writefile build/tongue/Quake_10x_Tongue/predictable.txt
cell_ontology_class
cell_type1
cluster
free_annotation

## Mouse aligned

In [ ]:
if not os.path.exists("build/tongue/ALIGNED_Mus_musculus_Tongue"):
    os.makedirs("build/tongue/ALIGNED_Mus_musculus_Tongue")
quake_smart_seq2_tongue = cb.data.ExprDataSet.read_dataset("../../Datasets/data/Quake_Smart-seq2_Tongue/data.h5")
quake_smart_seq2_tongue.obs["cluster"] = "cluster_" + quake_smart_seq2_tongue.obs["cluster"].astype(int).astype(str)
#quake_smart_seq2_tongue.obs["quake_smart_seq2_tongue_donor"] = \
#    quake_smart_seq2_tongue.obs["donor"]
#quake_10x_tongue.obs["quake_10x_tongue_donor"] = \
#    quake_10x_tongue.obs["donor"]
mouse = cb.data.ExprDataSet.merge_datasets(dict(
    quake_10x_tongue=quake_10x_tongue,
    quake_smart_seq2_tongue=quake_smart_seq2_tongue
), merge_uns_slots=["seurat_genes"])

In [ ]:
mouse.obs.head()

In [ ]:
mouse.obs.dtypes

In [ ]:
opt_model_kwargs = dict(
    batch_effect=["dataset_name", "donor"],
    #rmbatch_module=["MNNAdversarial", "Adversarial", "Adversarial", "Adversarial"],
    rmbatch_module_kwargs=dict(lambda_reg=0.01)
)
mouse_model = cb.directi.fit_DIRECTi(
    mouse, mouse.uns["seurat_genes"],
    **fixed_model_kwargs, **opt_model_kwargs
)
mouse.latent = mouse_model.inference(mouse)

In [ ]:
ax = mouse.visualize_latent("cell_ontology_class", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/tongue/ALIGNED_Mus_musculus_Tongue/cell_ontology_class.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = mouse.visualize_latent("dataset_name", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/tongue/ALIGNED_Mus_musculus_Tongue/dataset_name.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = mouse.visualize_latent("donor", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/tongue/ALIGNED_Mus_musculus_Tongue/donor.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = mouse.visualize_latent("gender", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/tongue/ALIGNED_Mus_musculus_Tongue/gender.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
mouse.write_dataset("build/tongue/ALIGNED_Mus_musculus_Tongue/ALIGNED_Mus_musculus_Tongue.h5")

In [ ]:
%%capture capio
mouse_models = [mouse_model]
opt_model_kwargs = dict(
    batch_effect=["dataset_name", "donor"],
    #rmbatch_module=["MNNAdversarial", "Adversarial", "Adversarial", "Adversarial"],
    rmbatch_module_kwargs=dict(lambda_reg=0.01)
)  # Many batch effects are being aligned, decrease regularization strength accordingly
for i in range(1, cb.config.N_JOBS):
    print("==== Model %d ====" % i)
    mouse_models.append(cb.directi.fit_DIRECTi(
        mouse, mouse.uns["seurat_genes"],
        **fixed_model_kwargs, **opt_model_kwargs,
        random_seed=i
    ))
mouse_blast = cb.blast.BLAST(mouse_models, mouse)
mouse_blast.save("build/tongue/ALIGNED_Mus_musculus_Tongue")

In [ ]:
with open("build/tongue/ALIGNED_Mus_musculus_Tongue/stdout.txt", "w") as f:
    f.write(capio.stdout)
with open("build/tongue/ALIGNED_Mus_musculus_Tongue/stderr.txt", "w") as f:
    f.write(capio.stderr)

In [ ]:
utils.self_projection(mouse_blast, "build/tongue/ALIGNED_Mus_musculus_Tongue")

In [ ]:
%%writefile build/tongue/ALIGNED_Mus_musculus_Tongue/predictable.txt
cell_ontology_class